## Pipeline de Transformación y Limpieza de Datos

### Crear dataset con datos que requieren transformación

In [1]:
import pandas as pd
import numpy as np

# Crear datos con problemas realistas
np.random.seed(42)
n = 1000

df = pd.DataFrame({
    'id_cliente': range(1, n+1),
    'edad': np.random.normal(35, 15, n).clip(18, 80).astype(int),
    'ingresos': np.random.lognormal(10, 0.8, n),
    'gastos_mensuales': np.random.normal(2000, 500, n).clip(500, 10000),
    'categoria_cliente': np.random.choice(['A', 'B', 'C', 'D'], n),
    'fecha_registro': pd.date_range('2020-01-01', periods=n, freq='D')[:n],
    'email': [f'cliente{i}@ejemplo.com' for i in range(1, n+1)],
    'telefono': [f'({np.random.randint(100, 999)}){np.random.randint(100, 999)}-{np.random.randint(1000, 9999)}' for _ in range(n)]
})

# Introducir algunos errores intencionalmente
error_indices = np.random.choice(n, 50, replace=False)
df.loc[error_indices[:20], 'edad'] = np.random.choice([-5, 150, np.nan], 20)  # Edades inválidas
df.loc[error_indices[20:35], 'ingresos'] = -1000  # Ingresos negativos
df.loc[error_indices[35:], 'gastos_mensuales'] = df.loc[error_indices[35:], 'ingresos'] * 2  # Gastos > ingresos

In [2]:
df.info() #observación inicial del df, en la cuenta ya se notan los nulos y se puede mirar si hay problemas en los tipos de datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_cliente         1000 non-null   int64         
 1   edad               992 non-null    float64       
 2   ingresos           1000 non-null   float64       
 3   gastos_mensuales   1000 non-null   float64       
 4   categoria_cliente  1000 non-null   object        
 5   fecha_registro     1000 non-null   datetime64[ns]
 6   email              1000 non-null   object        
 7   telefono           1000 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 62.6+ KB


In [3]:
df.describe() #Más información acerca de las categorías. aparece un número negativo como el valor mínimo de los ingresos y
# -5 con 150 son los valores límite de la edad

,id_cliente,edad,ingresos,gastos_mensuales,fecha_registro
count,1000.000000,992.000000,1000.000000,1000.000000,1000
mean,500.500000,36.564516,31531.958648,2712.432462,2021-05-14 12:00:00
min,1.000000,-5.000000,-1000.000000,500.000000,2020-01-01 00:00:00
25%,250.750000,25.000000,13350.632142,1682.002341,2020-09-06 18:00:00
50%,500.500000,35.000000,22810.256741,2005.334057,2021-05-14 12:00:00
75%,750.250000,44.000000,38927.926463,2340.766325,2022-01-19 06:00:00
max,1000.000000,150.000000,283363.570715,116643.085166,2022-09-26 00:00:00
std,288.819436,17.191621,29332.952117,7200.577985,NaN


In [4]:
df.duplicated().sum() #descarto registros duplicados

np.int64(0)

### Aplicar validaciones y correcciones

In [ ]:
# Validar y corregir edades
df['edad_valida'] = df['edad'].apply(lambda x: True if 18 <= x <= 80 else False)
df.loc[~df['edad_valida'], 'edad'] = np.nan  # Marcar inválidas como NaN
# ~ funciona como un not, así solicito los que tienen False con la sintaxis de pedir los True

# Validar ingresos (no negativos)
df.loc[df['ingresos'] < 0, 'ingresos'] = np.nan

# Validar gastos vs ingresos
df['ratio_gasto_ingreso'] = df['gastos_mensuales'] / df['ingresos']
df.loc[df['ratio_gasto_ingreso'] > 1, 'gastos_mensuales'] = df.loc[df['ratio_gasto_ingreso'] > 1, 'ingresos'] * 0.8

In [7]:
df.isnull().sum()

id_cliente              0
edad                   20
ingresos               15
gastos_mensuales        0
categoria_cliente       0
fecha_registro          0
email                   0
telefono                0
edad_valida             0
ratio_gasto_ingreso    15
dtype: int64

### Crear transformaciones y enriquecimientos:

In [ ]:
# Categorizar por edad
df['grupo_edad'] = pd.cut(df['edad'], 
                        bins=[18, 25, 35, 50, 80], 
                        labels=['Joven', 'Adulto_Joven', 'Adulto', 'Senior'], include_lowest=True) #uso include_lowest para que el primer bin 
                        #contenga a las personas con 18 años

# Calcular capacidad de ahorro
df['capacidad_ahorro'] = df['ingresos'] - df['gastos_mensuales']
df['ratio_ahorro'] = df['capacidad_ahorro'] / df['ingresos']

# Clasificar capacidad financiera
df['clasificacion_financiera'] = np.where(df['ratio_ahorro'] > 0.3, 'Ahorra_Mucho',
                                        np.where(df['ratio_ahorro'] > 0.1, 'Ahorra_Poco',
                                        np.where(df['ratio_ahorro'] > 0, 'Equilibra', 'Deficit')))

# Extraer información del teléfono
df['codigo_area'] = df['telefono'].str.extract(r'\((\d{3})\)')
# herramienta de str que selecciona (\d{3}) = 3 números entre parentesis, lo abren con \( y cierran con \)

# Calcular antigüedad
df['antiguedad_dias'] = (pd.Timestamp.now() - df['fecha_registro']).dt.days
df['antiguedad_meses'] = df['antiguedad_dias'] // 30

In [11]:
df

,id_cliente,edad,ingresos,gastos_mensuales,categoria_cliente,fecha_registro,email,telefono,edad_valida,ratio_gasto_ingreso,grupo_edad,capacidad_ahorro,ratio_ahorro,clasificacion_financiera,codigo_area,antiguedad_dias,antiguedad_meses
0,1,42.0,67473.104747,1662.410863,B,2020-01-01,cliente1@ejemplo.com,(138)588-4715,True,0.024638,Adulto,65810.693885,0.975362,Ahorra_Mucho,138,2171,72
1,2,32.0,46152.524769,1927.740665,D,2020-01-02,cliente2@ejemplo.com,(641)769-2301,True,0.041769,Adulto_Joven,44224.784105,0.958231,Ahorra_Mucho,641,2170,72
2,3,44.0,23102.688983,1603.790040,B,2020-01-03,cliente3@ejemplo.com,(302)165-3155,True,0.069420,Adulto,21498.898944,0.930580,Ahorra_Mucho,302,2169,72
3,4,57.0,13127.316641,1846.019235,B,2020-01-04,cliente4@ejemplo.com,(367)321-1692,True,0.140624,Senior,11281.297405,0.859376,Ahorra_Mucho,367,2168,72
4,5,31.0,38506.358054,1053.192667,A,2020-01-05,cliente5@ejemplo.com,(354)554-7396,True,0.027351,Adulto_Joven,37453.165387,0.972649,Ahorra_Mucho,354,2167,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,30.0,51850.517310,2038.740259,A,2022-09-22,cliente996@ejemplo.com,(106)739-6114,True,0.039320,Adulto_Joven,49811.777051,0.960680,Ahorra_Mucho,106,1176,39
996,997,61.0,21564.052962,2128.876270,A,2022-09-23,cliente997@ejemplo.com,(666)330-5335,True,0.098723,Senior,19435.176692,0.901277,Ahorra_Mucho,666,1175,39
997,998,44.0,10878.028272,1379.119712,C,2022-09-24,cliente998@ejemplo.com,(189)331-9648,True,0.126780,Adulto,9498.908560,0.873220,Ahorra_Mucho,189,1174,39
998,999,26.0,19332.568290,2167.088209,D,2022-09-25,cliente999@ejemplo.com,(332)453-6494,True,0.112095,Adulto_Joven,17165.480081,0.887905,Ahorra_Mucho,332,1173,39


### Crear métricas agregadas por categoría:

In [12]:
# Métricas por grupo de edad
metricas_edad = df.groupby('grupo_edad').agg({
    'ingresos': ['mean', 'median', 'std'],
    'capacidad_ahorro': 'mean',
    'ratio_ahorro': 'mean'
}).round(2)

print("Métricas por grupo de edad:")
print(metricas_edad)

# Resumen de validaciones
resumen_validacion = {
    'total_registros': len(df),
    'edades_invalidas': (~df['edad_valida']).sum(),
    'ingresos_negativos_corregidos': (df['ingresos'].isna()).sum(),
    'registros_procesados': len(df)
}

print("\nResumen de validación:")
for clave, valor in resumen_validacion.items():
    print(f"{clave}: {valor}")

Métricas por grupo de edad:
              ingresos                     capacidad_ahorro ratio_ahorro
                  mean    median       std             mean         mean
grupo_edad                                                              
Joven         33090.02  26240.77  26168.85         30704.12         0.88
Adulto_Joven  31465.15  23640.71  25554.76         29336.67         0.87
Adulto        31504.19  21448.63  32238.73         29145.95         0.87
Senior        31016.96  22286.82  26921.09         28876.55         0.88

Resumen de validación:
total_registros: 1000
edades_invalidas: 20
ingresos_negativos_corregidos: 15
registros_procesados: 1000


C:\Users\dlare\AppData\Local\Temp\ipykernel_14744\2255973119.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  metricas_edad = df.groupby('grupo_edad').agg({
